In [17]:
# 初始化库
from canlib import canlib, Frame
import struct

import matplotlib.pyplot as plt
import numpy as np

from PyQt5 import QtCore, QtWidgets

import time

# 交互式绘图后端
%matplotlib qt5
# %matplotlib widget


In [18]:
# 枚举端口
num_channels = canlib.getNumberOfChannels()
print(f"Found {num_channels} channels")
for ch in range(num_channels):
    chd = canlib.ChannelData(ch)
    print(f"{ch}. {chd.channel_name} ({chd.card_upc_no} / {chd.card_serial_no})")

# 比特率
cdbBitrate = canlib.Bitrate.BITRATE_1M

Found 3 channels
0. Kvaser Leaf Light v2 (channel 0) (73-30130-00685-0 / 113660)
1. Kvaser Virtual CAN Driver (channel 0) (00-00000-00000-0 / 0)
2. Kvaser Virtual CAN Driver (channel 1) (00-00000-00000-0 / 0)


# DUMP 功能


In [19]:
# 人工加入几个假变量
varTypeDict={}
varTypeDict["fakeFLoat1"] = "float"
varTypeDict["fakeFLoat2"] = "float"
varTypeDict["fakeFLoat3"] = "float"
varTypeDict["fakeFLoat4"] = "float"

In [20]:
# 数据准备
# 内置RAM CPU1版
startAddr = 0xC000
dumpSize = int(24 * 1024)
dumpSize = int(2 * 1000 * 1)
# 内置RAM CPU2版
# startAddr = 0x9000
# dumpSize = int(30 * 1024)
endAddr = startAddr+2*dumpSize

txFrameData = struct.pack("<I", startAddr) + struct.pack("<I", endAddr)

recvDict = {}

In [21]:
ch = canlib.openChannel(
    channel=0,
    flags=canlib.Open.EXCLUSIVE | canlib.Open.ACCEPT_VIRTUAL,
    bitrate=cdbBitrate,
)

# Set the CAN bus driver type to NORMAL.
ch.setBusOutputControl(canlib.Driver.NORMAL)

# Activate the CAN chip.
ch.busOn()

txFrame = Frame(id_=1, data=txFrameData, dlc=8)
ch.write(txFrame)

# 监听端口
timeout = 0.5
ticktime = 2
tick_countup = 0
frame_cnt = 0
frame_err = 0

while True:
    try:
        frame = ch.read(timeout=int(timeout * 1000))
        # 最基本的校验
        if (frame.flags != 2 or frame.id < 8 or frame.id > 15 or frame.dlc != 8):
            frame_err += 1
            print("收到{}个错误帧".format(frame_err))
            continue

        frame_cnt += 1
        if (frame_cnt % 1000 == 1):
            print("\r已收到{}个包".format(frame_cnt), end="")

        can_addr = struct.unpack("<I", frame.data[0:4])[0]

        # raw
        can_val = frame.data[4:8]

        recvDict[can_addr-startAddr] = can_val

        # debug
        # print("frame: ", frame_cnt)
        # print("addr: ", can_addr)
        # print("data val: ", can_val)

    except canlib.CanNoMsg:
        tick_countup += timeout

        if (frame_cnt == dumpSize):
            print("\nrecv ok!共{}个包".format(frame_cnt))
            break

        if (frame_cnt > dumpSize):
            print("\n收多了,有{}个包,寄!".format(frame_cnt))
            break

        if tick_countup > ticktime:
            print("\ntick")
            tick_countup = 0

    except KeyboardInterrupt:
        print("Stop.")
        break

# Inactivate the CAN chip.
ch.busOff()

# Close the channel.
ch.close()

已收到1001个包
recv ok!共2000个包


In [22]:
# 准备画图分析的数据

# 数据记录次序
# logTarNameList = ["CH1_Udc", "CH1_Udc_SI", "CH1_Id", "CH1_Iq", "targetId_CH1", "targetIq_CH1", "UdcAdrc.x1", "UdcAdrc.z1", "UdcAdrc.out_pi", "UdcAdrc.out_smc", "UdcAdrc.out_eso"]
# logTarNameList = ["CH1_Udc", "CH1_Id", "CH1_Iq", "targetId_CH1", "targetIq_CH1", "targetTe", "omegaMfbk", "thetaRDC_raw"]
# logTarNameList = ["CH1_Udc", "CH1_ifbk.V",
#                   "CH1_ifbk.W", "CH2_ifbk.U", "CH2_ifbk.V"]

# logTarNameList = ["CH1_Udc", "CH1_Id", "CH1_Iq", "targetId_CH1", "targetIq_CH1", "CH2_Id", "CH2_Iq"]

# logTarNameList = ["CH1_Udc", "CH1_ifbk.V", "CH2_ifbk.V", "targetTe"]

# logTarNameList = ["fakeFLoat1", "fakeFLoat2", "fakeFLoat3"]
logTarNameList = ["fakeFLoat1", "fakeFLoat2"]

totalVarCnt = len(logTarNameList)
totalCycle = int(dumpSize/totalVarCnt)

# timex = np.arange(totalCycle)
timex = np.arange(totalCycle)/1e3

valYnpTab = []

for ii in range(totalVarCnt):
    valYnpTab.append(np.zeros(totalCycle))
    valtype = varTypeDict.get(logTarNameList[ii], "float")

    if (valtype == "float"):
        for jj in range(totalCycle):
            valYnpTab[ii][jj] = struct.unpack(
                "<f", recvDict[(jj*totalVarCnt+ii)*2])[0]
    elif (valtype == "uint32_t"):
        for jj in range(totalCycle):
            valYnpTab[ii][jj] = struct.unpack(
                "<I", recvDict[(jj*totalVarCnt+ii)*2])[0]
    elif (valtype == "int32_t"):
        for jj in range(totalCycle):
            valYnpTab[ii][jj] = struct.unpack(
                "<i", recvDict[(jj*totalVarCnt+ii)*2])[0]
    elif (valtype == "uint16_t"):
        for jj in range(totalCycle):
            valYnpTab[ii][jj] = struct.unpack(
                "<H", recvDict[(jj*totalVarCnt+ii)*2][0:2])[0]
    elif (valtype == "int16_t"):
        for jj in range(totalCycle):
            valYnpTab[ii][jj] = struct.unpack(
                "<h", recvDict[(jj*totalVarCnt+ii)*2][0:2])[0]
    else:
        print("{}类型未知!".format(logTarNameList[ii]))

In [23]:
# 画图分析
drawNameList = []
# drawNameList.append("CH1_Udc_SI")
# drawNameList.append("CH1_Udc")
# drawNameList.append("CH1_Idc")
# drawNameList.append("CH1_Id")
# drawNameList.append("CH1_Iq")
# drawNameList.append("CH2_Id")
# drawNameList.append("CH2_Iq")
# drawNameList.append("targetId_CH1")
# drawNameList.append("targetIq_CH1")
# drawNameList.append("thetaRDC_raw")
# drawNameList.append("targetTe")
# drawNameList.append("omegaMfbk")
# drawNameList.append("CH1_ifbk.V")
# drawNameList.append("CH1_ifbk.W")
# drawNameList.append("CH2_ifbk.U")
# drawNameList.append("CH2_ifbk.V")
# drawNameList.append("fakeFLoat1")
drawNameList.append("fakeFLoat2")
# drawNameList.append("fakeFLoat3")

plt.figure()

for itemxName in drawNameList:
    for ii in range(totalVarCnt):
        if (logTarNameList[ii] != itemxName):
            continue
        plt.plot(timex, valYnpTab[ii], label=itemxName)

plt.xlabel("time:s")
plt.ylabel("value")
plt.title("value from dump mem")
# plt.xlim(100,200)
# plt.ylim(100,200)
plt.legend()
plt.show()

In [24]:
# 复制到剪贴板以便其他软件处理
# 不带首列

cbStr = ""

for ii in range(totalCycle):
    for jj in range(totalVarCnt-1):
        cbStr += str(valYnpTab[jj][ii])
        cbStr += "\t"
    cbStr += str(valYnpTab[totalVarCnt-1][ii])
    cbStr += "\n"

clipBoard = QtWidgets.QApplication.clipboard()
clipBoard.setText(cbStr)